# Extraction des données quotidiennes de températures d'ECCC et calculs d'indices

## Objectifs: 

L'objectif de ce produit est d'extraire les données quotidiennes de températures depuis le jeu de données homogénéisées de seconde génération d'Environnement et Changement Climatique Canada  développé par Vincent and al. 2012

Les données climatiques canadiennes ajustées et homogénéisées (DCCAH) ont été créées pour être utilisées dans les recherches climatiques en incluant les études des changements climatiques. Elles incluent un nombre d’ajustements aux données originales des stations pour traiter les sauts causés par les changements d’instruments et de procédures d’observations. Quelques fois les observations de plusieurs stations ont été combinées pour générer de longues séries temporelles.

Les données quotidiennes homogénéisées des températures maximales, minimales et moyennes and les données quotidiennes ajustées des chutes de pluie, de neige et de précipitation totale sont disponibles au <b>ftp://ccrp.tor.ec.gc.ca/pub/EC_data/AHCCD_daily/</b>.

 

Les observations originales du Service Météorologique de Canada sont disponibles sur http://climate.weather.gc.ca/historical_data/search_historic_data_f.html.


Dans notre étude nous allons extraire les données pour une station (en se référant au document: Temperature_Stations.xls) pour une période donnée (1988-2017).
Nous allons ensuite calculer plusieurs indices de températures (mensuels, saisonniers et annuels).

In [9]:
# importation des librairies 
import pandas as pd
import os
from datetime import date
import calendar
import numpy as np
import Indices_Temperatures
from dateutil.relativedelta import relativedelta
import pathlib
import warnings
warnings.filterwarnings("ignore")
from itertools import islice

La librairie locale <b>Indices_Temperatures</b>  regroupe plusieurs indices d'extrêmes inspirés de <a href="http://www.cru.uea.ac.uk/projects/stardex/" target="_blank">STARDEX 2004</a>. 

Nous allons travailler avec les données de température minimales journalières pour la station de Québec couvrant la période 1988-2018.

En se référant au document Temperature_Stations.xls, nous voyons que la station est <b>stnid = 701S001</b>.  

In [10]:
##########################-- Partie du code à modifier --######################
yearmin = 1988                                                            #
yearmax = 2018                                                            #
varin = 'dn'                                                                             
path = 'Homog_daily_min_temp_v2018'  
varout = 'Tasmin'                        
indice = Indices_Temperatures.MOY    # on va utiliser l'indice MOY 
indice_out = 'MOY'     

On va crée un dataframe avec la liste des stations d'ECCC. 

In [11]:
dataframe = pd.read_excel("./Temperature_Stations.xls", skiprows = range(0, 3))
dataframe.head()

,Prov,Nom de station,stnid,année déb.,mois déb.,année fin.,mois fin.,lat (deg),long (deg),élév (m),stns jointes
0,BC,AGASSIZ,1100120,1893,1,2018,9,49.25,-121.77,15,N
1,BC,ATLIN,1200560,1905,8,2018,12,59.57,-133.70,674,N
2,BC,BARKERVILLE,1090660,1888,2,2015,3,53.07,-121.52,1265,N
3,BC,BEAVERDELL,1130771,1939,1,2006,9,49.48,-119.05,838,Y
4,BC,BELLA COOLA,1060841,1895,5,2017,11,52.37,-126.68,18,Y


On va travailler avec la station de Québec: <b>stnid = 701S001</b>. On va extraire ici les données au format ascii de la station.

In [12]:
stnid = '701S001'   
f1 = open('./'+path+'/'+str(varin)+str(stnid)+'.txt', 'r')
for line in islice(f1, 10):
        print(line)

701S001,    QUEBEC         ,  QUE, station joined    , Homogenized daily minimum temperature        , Deg Celcius,          Updated to December 2018

701S001,    QUEBEC         ,  QUE, station jointe    , Temperature quotidienne minimale homogeneisee, Deg Celcius, Mise a jour jusqu a decembre 2018

 Year Mo  Day 01  Day 02  Day 03  Day 04  Day 05  Day 06  Day 07  Day 08  Day 09  Day 10  Day 11  Day 12  Day 13  Day 14  Day 15  Day 16  Day 17  Day 18  Day 19  Day 20  Day 21  Day 22  Day 23  Day 24  Day 25  Day 26  Day 27  Day 28  Day 29  Day 30  Day 31

Annee Mo Jour 01 Jour 02 Jour 03 Jour 04 Jour 05 Jour 06 Jour 07 Jour 08 Jour 09 Jour 10 Jour 11 Jour 12 Jour 13 Jour 14 Jour 15 Jour 16 Jour 17 Jour 18 Jour 19 Jour 20 Jour 21 Jour 22 Jour 23 Jour 24 Jour 25 Jour 26 Jour 27 Jour 28 Jour 29 Jour 30 Jour 31

 1875  8 -9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M-9999.9M   14.5a   15.8a   16.6a   13.9a   14.4a   17.3a   18.6a   18.6a   18.0a-9999.9M   12.4

### Nettoyage des données: 

On voit que dans notre jeu de données nous avons pour chaque ligne les données quotidiennes par année et par mois suivant la structure  :

<p style="color:blue;font-size:12px;"> Annee Mo Jour 01 Jour 02 Jour 03 Jour 04 Jour 05 Jour 06 Jour 07 Jour 08 Jour 09 Jour 10 Jour 11 Jour 12 Jour 13 Jour 14 Jour 15 Jour 16 Jour 17 Jour 18 Jour 19 Jour 20 Jour 21 Jour 22 Jour 23 Jour 24 Jour 25 Jour 26 Jour 27 Jour 28 Jour 29 Jour 30 Jour 31 </p>  

Nos fichiers ont un en-tête de 4 lignes que nous devrons supprimer.
On va également supprimer les caractères alphanumériques, nettoyer les valeurs manquantes et créer un dataframe.

In [13]:
f1 = open('./'+path+'/'+str(varin)+str(stnid)+'.txt', 'r')
f2 = open('./tmp.txt', 'w')
for line in f1:
    for word in line:
            if word == 'M':
                f2.write(word.replace('M', ' '))
            elif word == 'a':
                f2.write(word.replace('a', ' '))                    
            else:
                f2.write(word)
f1.close()
f2.close()      
df_station = pd.read_csv('./tmp.txt', delim_whitespace=True, skiprows = range(0, 4))
df_station.head()

,1875,8,-9999.9,-9999.9.1,-9999.9.2,-9999.9.3,-9999.9.4,-9999.9.5,-9999.9.6,-9999.9.7,...,12.4,7.6,8.3,13.9.1,13.9.2,19.8,17.3.1,13.5,14.4.1,15.9
0,1875,9,15.2,19.8,13.5,13.1,15.2,12.6,12.6,10.9,...,4.5,4.0,5.7,9.7,8.3,5.1,0.7,2.4,2.0,-9999.9
1,1875,10,3.4,2.4,-0.1,7.6,1.4,1.4,4.5,2.0,...,-3.6,2.0,1.4,2.0,-4.4,0.0,-5.0,-7.0,-2.8,0.0
2,1875,11,-4.4,-5.6,-5.0,-6.6,-5.0,-5.0,-6.6,-5.6,...,-14.2,-16.5,-8.9,-14.9,-10.8,-13.6,-18.2,-27.8,-27.8,-9999.9
3,1875,12,-27.8,-25.4,-20.8,-20.7,-11.4,-15.6,-14.9,-8.9,...,-5.6,-10.5,-19.1,-13.6,-13.6,-12.4,-14.2,-12.4,-8.6,2.4
4,1876,1,-2.8,-1.2,-1.2,-19.1,-20.1,-11.9,-14.2,-12.9,...,-24.1,-20.1,-20.8,-27.8,-18.3,-18.3,-9.8,-10.8,-24.8,-19.8


In [14]:
df_station.columns = ['Annee', 'Mois', 'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',
                                  'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20',
                                  'D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']
     
os.remove("./tmp.txt")
   
   # nettoyage des valeurs manquantes 
try:  
    df_station = df_station.replace({'E':''}, regex=True)
except:
       pass
try: 
    df_station = df_station.replace({'a':''}, regex=True)
except:
       pass
try:     
    df_station = df_station.replace({'-9999.9':''}, regex=True)
except:
       pass
try:     
    df_station = df_station.replace({-9999.9:''}, regex=True)
except:
       pass    
    
for col in  df_station.columns[2:]:
       df_station[col] = pd.to_numeric(df_station[col], errors='coerce')      

In [15]:
df_station.head()

,Annee,Mois,D1,D2,D3,D4,D5,D6,D7,D8,...,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31
0,1875,9,15.2,19.8,13.5,13.1,15.2,12.6,12.6,10.9,...,4.5,4.0,5.7,9.7,8.3,5.1,0.7,2.4,2.0,NaN
1,1875,10,3.4,2.4,-0.1,7.6,1.4,1.4,4.5,2.0,...,-3.6,2.0,1.4,2.0,-4.4,0.0,-5.0,-7.0,-2.8,0.0
2,1875,11,-4.4,-5.6,-5.0,-6.6,-5.0,-5.0,-6.6,-5.6,...,-14.2,-16.5,-8.9,-14.9,-10.8,-13.6,-18.2,-27.8,-27.8,NaN
3,1875,12,-27.8,-25.4,-20.8,-20.7,-11.4,-15.6,-14.9,-8.9,...,-5.6,-10.5,-19.1,-13.6,-13.6,-12.4,-14.2,-12.4,-8.6,2.4
4,1876,1,-2.8,-1.2,-1.2,-19.1,-20.1,-11.9,-14.2,-12.9,...,-24.1,-20.1,-20.8,-27.8,-18.3,-18.3,-9.8,-10.8,-24.8,-19.8


On peut maintenant détecter les années minimales et maximales d'enregistrement et écrire les données quotidienne sur une seule colonne. 

In [16]:
m_start =  df_station['Mois'].loc[(df_station['Annee'] == yearmin)].min()# premier mois 
m_end   =  df_station['Mois'].loc[(df_station['Annee'] == yearmax)].max()# dernier mois
d_end = calendar.monthrange(yearmax, m_end)[1]                     # nombre de jours d'enregistrement 

####################################Extraction des données quotidiennes  et ajout d'une colonne Date   
tmp_tmin = [ ] 
for year in range(yearmin,yearmax+1):    ### Boucle sur les annees
    for month in range(1,13):
        df = []
        last_day = calendar.monthrange(year, month)[1] 
        tmin = df_station.loc[(df_station["Annee"] == year) & (df_station["Mois"] == month)].iloc[:,2:last_day+2].values
           
        if len(tmin) == 0:
            a = np.empty((calendar.monthrange(year,month)[1]))
            a[:] = np.nan
            df=pd.DataFrame(a)
        else:
            df=pd.DataFrame(tmin.T)
               
        start = date(year, month, 1)
        end =   date(year, month, last_day)
        delta=(end-start) 
        nb_days = delta.days + 1 
        rng = pd.date_range(start, periods=nb_days, freq='D')          
        df['datetime'] = rng
        df.index = df['datetime']
        tmp_tmin.append(df)
           
tmp_tmin = pd.concat(tmp_tmin) 
df = pd.DataFrame({'datetime': tmp_tmin['datetime'], 'variable': tmp_tmin.iloc[:,0]}, columns = ['datetime','variable']) 
df.index = df['datetime'] 
df.head()

,datetime,variable
datetime,,
1988-01-01,1988-01-01,-9.6
1988-01-02,1988-01-02,-15.3
1988-01-03,1988-01-03,-10.2
1988-01-04,1988-01-04,-8.0
1988-01-05,1988-01-05,-19.3


## Calcul des indices mensuels

In [17]:
#################################### Calcul des indices mensuels     
resamp_tmin = df.resample('M').agg([indice])   
resamp_tmin.head()

,variable
,MOY
datetime,
1988-01-31,-15.261290
1988-02-29,-14.644828
1988-03-31,-8.993548
1988-04-30,1.046667
1988-05-31,8.193548


## Calcul des indices saisonniers

In [18]:
djf = []
mam = []
son=[]
jja=[]
incr= date(yearmin, 1, 1)
end = date(yearmax, 12, 31)
while incr <= end:
            current_year = str(incr.year)
            last_year = str(incr.year-1)
            try:
                dec = df[last_year][np.in1d(df[last_year].index.month, [12])]
            except:
                rng = pd.date_range(last_year, periods=31, freq='D')
                dec = pd.DataFrame({'datetime': rng, 'variable': [np.nan]*31}, columns = ['datetime','variable']) 
                
            j_f = df[current_year][np.in1d(df[current_year].index.month, [1,2])]
               
            djf.append(indice(dec.append(j_f).iloc[:,1].values))
            
            mam.append(indice((df[current_year][np.in1d(df[current_year].index.month, [3,4,5])].iloc[:,1]).values))
            jja.append(indice((df[current_year][np.in1d(df[current_year].index.month, [6,4,8])].iloc[:,1]).values))
            son.append(indice((df[current_year][np.in1d(df[current_year].index.month, [9,10,11])].iloc[:,1]).values))
              
            incr = incr + relativedelta(years=1)
TIME=[]
for y in range(yearmin,yearmax+1,1):
        TIME.append(y)
    
df_djf = pd.DataFrame({'Date': TIME,'Indice DJF': djf}, columns = ['Date','Indice DJF']) 
df_mam = pd.DataFrame({'Date': TIME,'Indice MAM': mam}, columns = ['Date','Indice MAM']) 
df_jja = pd.DataFrame({'Date': TIME,'Indice JJA': jja}, columns = ['Date','Indice JJA']) 
df_son = pd.DataFrame({'Date': TIME,'Indice SON': son}, columns = ['Date','Indice SON']) 

Exemple d'indices saisonniers pour l'hiver:

In [19]:
df_djf.head() 

,Date,Indice DJF
0,1988,NaN
1,1989,-15.526667
2,1990,-15.381111
3,1991,-13.867778
4,1992,-15.431868


## Calcul des indices annuels

In [20]:
annual = []
df_annual = []
incr= date(yearmin, 1, 1)
end = date(yearmax, 12, 31)
while incr <= end:
     current_year = str(incr.year)
     annual.append(indice(df[current_year].iloc[:,1].values))         
     incr = incr + relativedelta(years=1)
TIME=[]
for y in range(yearmin,yearmax+1,1):
    TIME.append(y)
            
df_annual = pd.DataFrame({'Date': TIME,'Indice Annuel': annual}, columns = ['Date','Indice Annuel']) 
df_annual.head()

,Date,Indice Annuel
0,1988,0.727869
1,1989,-0.609041
2,1990,1.410959
3,1991,0.930137
4,1992,-0.130328


## Code final

Le code qui suit permet d'extraire les données quotidiennes pour une station, de calculer les indices suivanat plusieurs fréquences et de sauvegarder les sorties au format csv.

In [21]:
##########################-- Partie du code à modifier --######################
yearmin = 1988                                                              #
yearmax = 2018                                                             #
varin = 'dn'                         # dx, dn, dm                                                           
path = 'Homog_daily_min_temp_v2018'  #'Homog_daily_max_temp_v2018'   'Homog_daily_min_temp_v2018'  'Homog_daily_mean_temp_v2018'                                            
varout = 'Tasmin'                    # Tasmax, Tasmin, Tasmoy      
list_indices = ['MOY','Tmin10p','Tmax90p']
name = 'Quebec'
stnid = '701S001'
###############################################################################
for ind in list_indices:    
    if ind == 'MOY':
       indice = Indices_Temperatures.MOY  
       indice_out = 'MOY'     
    elif ind == 'Tmax90p':
       indice = Indices_Temperatures.Tmax90p
       indice_out = 'Tmax90p'
    elif ind == 'Tmin10p':
       indice = Indices_Temperatures.Tmin10p
       indice_out = 'Tmin10p'
       
    ### lecture de toutes les stations d'ECCC pour la température
    dataframe = pd.read_excel("./Temperature_Stations.xls", skiprows = range(0, 3))
    
    ### on va filtrer les données ayant pour période commune: yearmin - yearmax
    globals()['dataframe_'+str(yearmin)+'_'+str(yearmax)] = dataframe.loc[(dataframe["année déb."] <= yearmin) & (dataframe["année fin."] >= yearmax),:]
       
    f1 = open('./'+path+'/'+str(varin)+str(stnid)+'.txt', 'r')
    f2 = open('./tmp.txt', 'w')
    
    ### nettoyage des données   
    for line in f1:
            for word in line:
                if word == 'M':
                    f2.write(word.replace('M', ' '))
                elif word == 'a':
                    f2.write(word.replace('a', ' '))                    
                else:
                    f2.write(word)
    f1.close()
    f2.close()
              
    station = pd.read_csv('./tmp.txt', delim_whitespace=True, skiprows = range(0, 4))
       
    station.columns = ['Annee', 'Mois', 'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',
                                      'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20',
                                      'D21','D22','D23','D24','D25','D26','D27','D28','D29','D30','D31']
         
    os.remove("./tmp.txt")
       
       # nettoyage des valeurs manquantes 
    try:  
           station = station.replace({'E':''}, regex=True)
    except:
           pass
    try: 
           station = station.replace({'a':''}, regex=True)
    except:
           pass
    try:     
           station = station.replace({'-9999.9':''}, regex=True)
    except:
           pass
    try:     
           station = station.replace({-9999.9:''}, regex=True)
    except:
           pass    
           
    for col in  station.columns[2:]:
           station[col] = pd.to_numeric(station[col], errors='coerce')
           
    m_start =  station['Mois'].loc[(station['Annee'] == yearmin)].min()
    m_end   =  station['Mois'].loc[(station['Annee'] == yearmax)].max()
       
    d_end = calendar.monthrange(yearmax, m_end)[1]
       
    ####################################Extraction des données quotidiennes  et ajout d'une colonne Date      
    tmp_tmin = [ ] 
    for year in range(yearmin,yearmax+1):    ### Boucle sur les annees
           for month in range(1,13):
               df = []
               last_day = calendar.monthrange(year, month)[1] 
               tmin = station.loc[(station["Annee"] == year) & (station["Mois"] == month)].iloc[:,2:last_day+2].values
               
               if len(tmin) == 0:
                   a = np.empty((calendar.monthrange(year,month)[1]))
                   a[:] = np.nan
                   df=pd.DataFrame(a)
               else:
                   df=pd.DataFrame(tmin.T)
                   
               start = date(year, month, 1)
               end =   date(year, month, last_day)
               delta=(end-start) 
               nb_days = delta.days + 1 
               rng = pd.date_range(start, periods=nb_days, freq='D')          
               df['datetime'] = rng
               df.index = df['datetime']
               tmp_tmin.append(df)
               
    tmp_tmin = pd.concat(tmp_tmin) 
    df = pd.DataFrame({'datetime': tmp_tmin['datetime'], 'variable': tmp_tmin.iloc[:,0]}, columns = ['datetime','variable']) 
    df.index = df['datetime']  
    #################################### Calcul des indices mensuels     
    resamp_tmin = df.resample('M').agg([indice ])       
    #################################### Calcul des indices saisonniers  
       
    djf = []
    mam = []
    son=[]
    jja=[]
    incr= date(yearmin, 1, 1)
    end = date(yearmax, 12, 31)
    while incr <= end:
            current_year = str(incr.year)
            last_year = str(incr.year-1)
            try:
                dec = df[last_year][np.in1d(df[last_year].index.month, [12])]
            except:
                rng = pd.date_range(last_year, periods=31, freq='D')
                dec = pd.DataFrame({'datetime': rng, 'variable': [np.nan]*31}, columns = ['datetime','variable']) 
                
            j_f = df[current_year][np.in1d(df[current_year].index.month, [1,2])]
               
            djf.append(indice(dec.append(j_f).iloc[:,1].values))
            
            mam.append(indice((df[current_year][np.in1d(df[current_year].index.month, [3,4,5])].iloc[:,1]).values))
            jja.append(indice((df[current_year][np.in1d(df[current_year].index.month, [6,4,8])].iloc[:,1]).values))
            son.append(indice((df[current_year][np.in1d(df[current_year].index.month, [9,10,11])].iloc[:,1]).values))
              
            incr = incr + relativedelta(years=1)
    
    #################################### Calcul des indices annuels  
    annual = []
    df_annual = []
    incr= date(yearmin, 1, 1)
    end = date(yearmax, 12, 31)
    while incr <= end:
            current_year = str(incr.year)
            annual.append(indice(df[current_year].iloc[:,1].values))         
            incr = incr + relativedelta(years=1)
            
    #################################### Écriture des indices en format csv     
    TIME=[]
    for y in range(yearmin,yearmax+1,1):
            TIME.append(y) 
    df_annual = pd.DataFrame({'Date': TIME,'Indice': annual}, columns = ['Date','Indice Annuel']) 
            
    #################################### Écriture des indices en format csv 
    outpath='./INDICES/'
    pathlib.Path(outpath).mkdir(parents=True, exist_ok=True)
       
    TIME=[]
    for y in range(yearmin,yearmax+1,1):
            TIME.append(y)
            
    df_annual = pd.DataFrame({'Date': TIME,'Indice Annuel': annual}, columns = ['Date','Indice Annuel']) 
         
    df_djf = pd.DataFrame({'Date': TIME,'Indice DJF': djf}, columns = ['Date','Indice DJF']) 
    df_mam = pd.DataFrame({'Date': TIME,'Indice MAM': mam}, columns = ['Date','Indice MAM']) 
    df_jja = pd.DataFrame({'Date': TIME,'Indice JJA': jja}, columns = ['Date','Indice JJA']) 
    df_son = pd.DataFrame({'Date': TIME,'Indice SON': son}, columns = ['Date','Indice SON']) 

    df_annual.to_csv(outpath+name+'_SEASON_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_YEAR.csv')
      
    df_djf.to_csv(outpath+name+'_SEASON_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_DJF.csv')
    df_mam.to_csv(outpath+name+'_SEASON_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_MAM.csv')
    df_jja.to_csv(outpath+name+'_SEASON_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_JJA.csv')
    df_son.to_csv(outpath+name+'_SEASON_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_SON.csv') 
          
    for m in range(1,13):
           month_var = resamp_tmin[resamp_tmin.index.month==m]
           month_var.to_csv(outpath+name+'_MONTH_'+varout+'_'+indice_out+'_'+str(yearmin)+'_'+str(yearmax)+'_'+str("{:02}".format(m))+'.csv')
    